In [ ]:
import pycaret.classification as pc
import pandas as pd
from sklearn.metrics import log_loss
import mlflow
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

X_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_test.parquet")
y_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_test.parquet")

X_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_train.parquet")
y_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_train.parquet")
data_train = pd.concat([X_train, y_train], axis=1)

mlflow.set_tracking_uri('http://localhost:5000/')

EXPERIMENT_NAME = "kobe"
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

tags = {
        "Projeto": "22E1_3_Kobe",
        "team": "Daniel S. U. Tamashiro",
        "dataset": "Kobe Bryant Shot Selection"
       }

with mlflow.start_run(experiment_id=experiment, run_name="Treinamento") as run:
    # Log pipeline parameters to MLflow
    #mlflow.log_param("scaler", "StandardScaler")
    mlflow.log_param("classifier", "LogisticRegression")

    clf = pc.setup(data = data_train, target = "shot_made_flag")
    lr = pc.create_model('lr')
    #pipe = Pipeline([
        #('scaler', StandardScaler()),
        #('lr', lr)
    #])
    tuned_pipe = pc.tune_model(lr)

    pc.plot_model(tuned_pipe, plot='auc')
    temp_name = "auc.png"
    plt.savefig(temp_name)
    mlflow.log_artifact(temp_name, "Curva ROC")

    y_pred = tuned_pipe.predict_proba(X_test)
    log_loss_score = log_loss(y_test, y_pred)
    print('Log loss score: {:.4f}'.format(log_loss_score))

    # Track tags
    mlflow.set_tags(tags)

    # Track parameters
    mlflow.log_param("target", "shot_made_flag")
    mlflow.log_param("model", "lr")

    # Track metrics
    mlflow.log_metric("log_loss", log_loss_score)

    # Track model
    mlflow.sklearn.log_model(tuned_pipe, "model")
    
    #Registro do modelo
    mlflow.log_artifact(local_path='./model.ipynb',artifact_path='code')
    

mlflow.end_run()



In [1]:
import pycaret.classification as pc
import pandas as pd
from sklearn.metrics import log_loss
import mlflow
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

X_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_test.parquet")
y_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_test.parquet")

X_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_train.parquet")
y_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_train.parquet")
data_train = pd.concat([X_train, y_train], axis=1)

In [4]:
data_train.head

<bound method NDFrame.head of            lat       lon  minutes_remaining  period  playoffs  shot_distance  \
0      34.0343 -118.1288                  1       2         0             14   
1      33.9723 -118.0808                  0       3         0             20   
2      33.8713 -118.1728                 11       3         0             19   
3      34.0443 -118.2698                 11       2         0              0   
4      33.8543 -118.2698                 11       3         0             19   
...        ...       ...                ...     ...       ...            ...   
16223  34.0523 -118.0718                  9       3         1             19   
16224  34.0443 -118.2698                  0       1         0              0   
16225  34.0523 -118.0838                  2       3         0             18   
16226  34.0403 -118.3618                  3       3         0              9   
16227  33.8573 -118.3958                  6       1         1             22   

       sh

In [6]:
import pyarrow.parquet as pq
import pandas as pd


data_train = pd.concat([X_train, y_train], axis=1)
print(data_train)


clf = pc.setup(data = data_train, target = "shot_made_flag")
lr = pc.create_model('lr')
#pipe = Pipeline([
#('scaler', StandardScaler()),
#('lr', lr)
#])
tuned_pipe = pc.tune_model(lr)

           lat       lon  minutes_remaining  period  playoffs  shot_distance  \
0      34.0343 -118.1288                  1       2         0             14   
1      33.9723 -118.0808                  0       3         0             20   
2      33.8713 -118.1728                 11       3         0             19   
3      34.0443 -118.2698                 11       2         0              0   
4      33.8543 -118.2698                 11       3         0             19   
...        ...       ...                ...     ...       ...            ...   
16223  34.0523 -118.0718                  9       3         1             19   
16224  34.0443 -118.2698                  0       1         0              0   
16225  34.0523 -118.0838                  2       3         0             18   
16226  34.0403 -118.3618                  3       3         0              9   
16227  33.8573 -118.3958                  6       1         1             22   

       shot_made_flag  
0              

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:13:11
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5590,0.5753,0.4843,0.5434,0.5122,0.1122,0.1128
1,0.5915,0.6161,0.4862,0.5880,0.5323,0.1755,0.1780
2,0.5951,0.6202,0.4945,0.5903,0.5382,0.1826,0.1849
3,0.5731,0.5874,0.4779,0.5618,0.5165,0.1387,0.1402
4,0.5739,0.6004,0.4723,0.5639,0.5141,0.1400,0.1417
5,0.5643,0.5753,0.4723,0.5505,0.5084,0.1212,0.1224
6,0.5968,0.6083,0.5166,0.5882,0.5501,0.1876,0.1889
7,0.5907,0.6151,0.4871,0.5854,0.5317,0.1735,0.1759
8,0.5687,0.5756,0.5000,0.5531,0.5252,0.1319,0.1324


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5590,0.5753,0.4843,0.5434,0.5122,0.1122,0.1128
1,0.5915,0.6161,0.4862,0.5880,0.5323,0.1755,0.1780
2,0.5951,0.6202,0.4945,0.5903,0.5382,0.1826,0.1849
3,0.5731,0.5874,0.4779,0.5618,0.5165,0.1387,0.1402
4,0.5739,0.6004,0.4723,0.5639,0.5141,0.1400,0.1417
5,0.5643,0.5753,0.4723,0.5505,0.5084,0.1212,0.1224
6,0.5968,0.6083,0.5166,0.5882,0.5501,0.1876,0.1889
7,0.5907,0.6151,0.4871,0.5854,0.5317,0.1735,0.1759
8,0.5687,0.5756,0.5000,0.5531,0.5252,0.1319,0.1324


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [16]:
metrics = clf.get_metrics()
#convert the Pandas Series object to a dictionary
#metrics= metrics.to_dict() 
type(metrics)

pandas.core.frame.DataFrame

In [16]:
#classification model

clf = pc.setup(data = data_train, target = "shot_made_flag")
models = pc.compare_models(sort='AUC', n_select=5)

# Print the names of the best 5 models
for model in models:
    model_name = str(type(model)).split(".")[-1][:-2]
    print(model_name)

,Description,Value
0,Session id,1258
1,Target,shot_made_flag
2,Target type,Binary
3,Original data shape,"(16228, 7)"
4,Transformed data shape,"(16228, 7)"
5,Transformed train set shape,"(11359, 7)"
6,Transformed test set shape,"(4869, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


LinearDiscriminantAnalysis
QuadraticDiscriminantAnalysis
LogisticRegression
GaussianNB
AdaBoostClassifier


In [15]:
for model in models:
    model_name = str(type(model)).split(".")[-1][:-2]
    print(model_name)
    

AdaBoostClassifier
QuadraticDiscriminantAnalysis
LinearDiscriminantAnalysis
GradientBoostingClassifier
LogisticRegression


In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
import numpy as np

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_score = lda.decision_function(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# calculate F1-score for each threshold
f1_scores = []
for threshold in thresholds:
    y_pred = y_score > threshold
    f1 = f1_score(y_test, y_pred)
    f1_scores.append(f1)

# find the index of the threshold that maximizes F1-score
best_threshold_index = np.argmax(f1_scores)

# get the best threshold and its corresponding F1-score
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]


# print the best threshold and its corresponding F1-score
print("Best threshold:", best_threshold)
print("Best F1-score:", best_f1_score)

Best threshold: -0.7662229156607268
Best F1-score: 0.6468513853904282
